# Grab all the loopy/inlining/odd shapes
Select a couple of examples to see what the pattern is for `stop_sequence` and `shape_meters`.

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'

import dask.dataframe as dd
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely
import sys

from segment_speed_utils import (helpers, gtfs_schedule_wrangling, 
                                 wrangle_shapes)
from segment_speed_utils.project_vars import SEGMENT_GCS, analysis_date

# Temp append so we can import functions from scripts/
sys.path.append("./scripts/")
import prep_stop_segments
import cut_stop_segments
import loop_utils
import loopy

shape_with_utc = "f765b9d12fcca0173b4e3ddbc0374d18"
another_shape = "007cf76062f5957d4e38ea54e624c0ad"



In [ ]:
loop_shapes = loop_utils.grab_loop_shapes(analysis_date)

stop_times_with_geom = prep_stop_segments.stop_times_aggregated_to_shape_array_key(
        analysis_date, loop_shapes)

st_loops = stop_times_with_geom.compute()

In [ ]:
gdf = (loop_utils.assign_visits_to_stop(st_loops)
       .sort_values(["shape_array_key", "stop_sequence"])
       .reset_index(drop=True)
      )

In [ ]:
gdf2 = gdf[gdf.shape_array_key.isin([#shape_with_utc, 
    another_shape])]

In [ ]:
gdf2.head()

In [ ]:
def get_shape_components(
    shape_geometry: shapely.geometry.LineString,
) -> tuple:
    """
    For a shape, we want to get the list of shapely.Points and
    a calculated cumulative distance array.
    """
    shape_coords_list = [shapely.Point(i) for 
                         i in shape_geometry.simplify(0).coords]
    
    # calculate the distance between current point and prior
    # need to remove the first point so that we can 
    # compare to the prior
    point_series_no_idx0 = loopy.array_to_geoseries(
        shape_coords_list[1:],
        geom_type="point"
    )

    points_series = loopy.array_to_geoseries(
        shape_coords_list, 
        geom_type="point"
    )
    
    distance_from_prior = np.array(
        point_series_no_idx0.distance(points_series)
    )
    
    # Based on distance_from_prior, now create a 
    # cumulative distance array, and append 0 to 
    # the beginning. We want length of this array to match the 
    # length of stop_sequence array
    cumulative_distances = np.array(
        [0] + list(np.cumsum(distance_from_prior))
    )
    
    return shape_coords_list, cumulative_distances

In [ ]:
def super_project(
    current_stop_seq: int,
    shape_geometry: shapely.geometry.LineString,
    stop_geometry_array: np.ndarray,
    stop_sequence_array: np.ndarray,
):
    
    shape_coords_list, cumulative_distances = get_shape_components(
        shape_geometry)

    # (1) Given a stop sequence value, find the stop_sequence values 
    # just flanking it (prior and subsequent).
    # this is important especially because stop_sequence does not have 
    # to be increasing in increments of 1, but it has to be monotonically increasing
    subset_seq = loopy.include_prior(
        stop_sequence_array, current_stop_seq)
    
    #https://stackoverflow.com/questions/31789187/find-indices-of-large-array-if-it-contains-values-in-smaller-array
    idx_stop_seq = np.where(np.in1d(
        stop_sequence_array, subset_seq))[0]
    
    # (2) Grab relevant subset based on stop sequence values to get stop geometry subset
    # https://stackoverflow.com/questions/5508352/indexing-numpy-array-with-another-numpy-array    
    subset_stop_geom = stop_geometry_array[
        idx_stop_seq[0]:
        idx_stop_seq[-1] + 1
    ]
    
    
    # (3a) Project this vector of start/end stops
    subset_stop_proj = loopy.project_list_of_coords(
        shape_geometry, subset_stop_geom)
    
    # 2 cases, at stop 1, we need to manually set the starting point
    # to be the start of the shape, and the distance is just
    # distance from start of shape to stop1.
    if len(subset_stop_proj) == 1:
        start_stop = 0
        end_stop = subset_stop_proj[0]
        distance_between_stops = subset_stop_proj[0]
        
    # The normal case is that there is a prior stop and a distance
    # elapsed between the current and prior stop
    else:
        # use this to get sense of direction, whether 
        # distance is increasing or decreasing as we move from prior to current stop
        start_stop = subset_stop_proj[0]
        end_stop = subset_stop_proj[-1]
        
        # (3b) Calculate distance between stops
        distance_between_stops = subset_stop_geom[0].distance(
            subset_stop_geom[-1])
    
    # (4) We know distance between stops, so let's back out the correct
    # "end_stop". If the end_stop is actually going 
    # back closer to the start of the shape, we'll use 
    # subtraction.
    
    # Normal case
    if start_stop < end_stop:
        origin_stop = start_stop
        destin_stop = start_stop + distance_between_stops
        
        # change this to point
        origin_destination_geom = loopy.interpolate_projected_points(
            shape_geometry, [origin_stop, destin_stop])
    
    # Case where inlining occurs, and now the bus is doubling back    
    elif start_stop > end_stop:
        origin_stop = start_stop
        destin_stop = start_stop - distance_between_stops
        
        # Flip this so when we order the subset of points, we 
        # correctly append the origin to the shape_coords_list 
        # (which is destination, since it has a lower value)
        origin_destination_geom = loopy.interpolate_projected_points(
            shape_geometry, [destin_stop, origin_stop])
    
    # Case at origin, where there is no prior stop to look for
    elif start_stop == end_stop:
        origin_stop = 0
        destin_stop = start_stop
        
        origin_destination_geom = loopy.interpolate_projected_points(
            shape_geometry, [origin_stop, destin_stop]
        )
        
    # (5) Find the subset from cumulative distances
    # that is in between our origin stop and destination stop
    shape_dist_subset_indices = loopy.cut_shape_by_origin_destination(
        cumulative_distances,
        (origin_stop, destin_stop)
    )
        
    # Now grab these indices from our coordinates list
    upper_idx = shape_dist_subset_indices[-1]+1
    
    if upper_idx >len(cumulative_distances):
        subset_shape_geom = shape_coords_list[
            shape_dist_subset_indices[0]: 
            upper_idx-1
        ]
    
    else:
        subset_shape_geom = shape_coords_list[
            shape_dist_subset_indices[0]: 
            upper_idx
        ]
    
    # Attach the origin and destination, otherwise the segment
    # will not reach the actual stops, but will just grab the trunk portion
    subset_shape_geom_with_od = np.array(
        [origin_destination_geom[0]] + 
        subset_shape_geom + 
        [origin_destination_geom[-1]]
    )
    
    return subset_shape_geom_with_od#, origin_destination_geom

In [ ]:
def stop_segment_components_to_geoseries(
    subset_shape_geom_array: np.ndarray,
    subset_stop_geom_array: np.ndarray = [],
    crs: str = "EPSG:3310"
) -> tuple:#[gpd.GeoDataFrame]:
    """
    Turn segments and stops into geoseries so we can plot it easily.
    """
    stop_segment = loopy.array_to_geoseries(
        subset_shape_geom_array, 
        geom_type="line",
        crs=crs
    )#.to_frame(name="stop_segment")
    
    if len(subset_stop_geom_array) > 0:
        related_stops = loopy.array_to_geoseries(
            subset_stop_geom_array,
            geom_type="point",
            crs=crs
        )#.to_frame(name="surrounding_stops_geom")

        return stop_segment, related_stops
    else:
        return stop_segment

In [ ]:
def plot_segments_and_stops(
    segment: gpd.GeoSeries, 
    stops: gpd.GeoSeries
):
    m = segment.explore(tiles="CartoDB Positron", name="segment")
    m = stops.explore(m=m, name="stops")

    folium.LayerControl().add_to(m)
    return m

In [ ]:
shape_geometry = gdf2.geometry.iloc[0]
stop_geometry_array = np.array(gdf2.stop_geometry)
stop_sequence_array = np.array(gdf2.stop_sequence)

In [ ]:
current_stop_seq = 33

test_segment, test_stops = super_project(
    current_stop_seq,
    shape_geometry,
    stop_geometry_array,
    stop_sequence_array,
)

segment_to_plot, stops_to_plot = stop_segment_components_to_geoseries(
    test_segment, test_stops)

In [ ]:
plot_segments_and_stops(segment_to_plot, stops_to_plot)

## Expand to multiple stops within same shape

In [ ]:
subset_df = gdf2[gdf2.shape_array_key==another_shape
                ].reset_index(drop=True)

shape_geometry = subset_df.geometry.iloc[0]
stop_geometry_array = np.array(subset_df.stop_geometry)
stop_sequence_array = np.array(subset_df.stop_sequence)

segment_results = []

In [ ]:
for stop_seq in stop_sequence_array:
        
    segment_i = super_project(
        stop_seq,
        shape_geometry,
        stop_geometry_array,
        stop_sequence_array,
    )
    segment_i_series = stop_segment_components_to_geoseries(
        segment_i, [])
    
    segment_results.append(segment_i_series)


In [ ]:
stop_segments = pd.concat(segment_results, axis=0)

In [ ]:
stop_segments.to_frame(name="stop_segments").head()

In [ ]:
results_df = gpd.GeoDataFrame()

for one_shape in gdf3.shape_array_key.unique():
    subset_df = gdf3[gdf3.shape_array_key == one_shape
                    ].reset_index(drop=True)
    
    shape_geometry = subset_df.geometry.iloc[0]
    stop_geometry_array = np.array(subset_df.stop_geometry)
    stop_sequence_array = np.array(subset_df.stop_sequence)
    
    segment_results = []
    surrounding_stops = []
    
    for stop_seq in stop_sequence_array:
        segment_i, stops_i = super_project(
        stop_seq,
        shape_geometry,
        stop_geometry_array,
        stop_sequence_array,
    )
    segment_i_series, stops_i_series = stop_segment_components_to_geoseries(
        segment_i, stops_i)
    
    segment_results.append(segment_i_series)
    surrounding_stops.append(stops_i_series)
    
    subset_df = subset_df.assign(
        stop_segment = segment_results,
        surrounding_stops_geom = surrounding_stops
    )
    
    results_df = pd.concat([results_df, subset_df], axis=0).reset_index(drop=True)

In [ ]:
shape_geometry = gdf2.geometry.iloc[0]
stop_geometry_array = np.array(gdf2.stop_geometry)
stop_sequence_array = np.array(gdf2.stop_sequence)

current_stop_seq = 13

test_segment, test_stops = super_project(
    current_stop_seq,
    shape_geometry,
    stop_geometry_array,
    stop_sequence_array,
)


segment_to_plot, stops_to_plot = stop_segment_components_to_geoseries(
    test_segment, test_stops)